# Catchment area analysis of supermarkets

## Libraries and settings

In [17]:
# Libraries
import os
import time
import glob
import json
import folium
import requests
import platform
import numpy as np
import pandas as pd
import geopandas as gdp
import matplotlib.pyplot as plt
from openrouteservice import client
from IPython.display import clear_output
from IPython.display import display, HTML

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import supermarket data

In [4]:
# Read OpenRouteService API key
with open(file='ors_token.txt', mode='r') as file:
    api_key = file.read()
# print(api_key)

# Import supermarket data
df_orig = pd.read_csv('supermarkets_data_enriched.csv', 
                      sep=',',
                      encoding='utf-8')[['id',
                                         'lat',
                                         'lon',
                                         'brand',
                                         'bfs_number',
                                         'bfs_name',
                                         'addr:housenumber',
                                         'addr:postcode',
                                         'geometry']]

# Subset, only supermarkets with complete address
df = df_orig.dropna()
print(df.shape)
df.head()

(967, 9)


,id,lat,lon,brand,bfs_number,bfs_name,addr:housenumber,addr:postcode,geometry
1,280130028,47.155492,9.039666,ALDI,3315,Schänis,3,8718.0,POINT (9.0396663 47.1554921)
2,6122906632,47.158959,9.044477,Denner,3315,Schänis,14,8718.0,POINT (9.0444769 47.1589589)
4,36726161,47.226191,8.980329,Migros,3339,Uznach,25,8730.0,POINT (8.9803292 47.2261912)
6,7389830915,47.225056,8.970650,Denner,3339,Uznach,15,8730.0,POINT (8.9706499 47.2250563)
7,39947904,47.376732,8.542161,Coop,261,Zürich,1,8001.0,POINT (8.5421608 47.3767316)


## Create isochrone for a single supermarket

In [24]:
# Settings
municip = 'Winterthur'
transport = 'driving-car' # alternativle 'foot-walking'
traveltime = 20 # in minutes

# Client settings
ors = client.Client(key=api_key)

# Set up supermarket dictionary with a single supermarket
df_sub = df.loc[df['bfs_name'] == municip].iloc[0]
supermarkets = {df_sub['brand']: {'location': [df_sub['lon'], 
                                            df_sub['lat']]}}
print(supermarkets)

# Set up folium map
map = folium.Map(tiles='openstreetmap', 
                location=([df_sub['lat'], 
                            df_sub['lon']]), 
                zoom_start=11)

# Parameters for server-request
params_iso = {'profile': transport,
            'range': [traveltime*60], # in seconds
            'attributes': ['total_pop']}

# Server request
for name, apt in supermarkets.items():
    
    # Add coords to request parameters
    params_iso['locations'] = [apt['location']] 
    
    # Perform isochrone request
    apt['iso'] = ors.isochrones(**params_iso)
    
    # Add GeoJson to map
    folium.features.GeoJson(apt['iso']).add_to(map)
    
    # Save GeoJson as file
    # with open(f'{name}.json', 'w') as f:
        # f.write(json.dumps(apt['iso']))

    # Reverse coords due to weird folium lat/lon syntax
    folium.map.Marker(list(reversed(apt['location'])),
                    icon = folium.Icon(color='green',
                                        icon_color='#cc0000',
                                        icon='home',
                                        prefix="fa"),
                    popup = name
                    ).add_to(map)

# Plot map
map

{'Migros': {'location': [8.7064484, 47.4918744]}}


## Getting the number of residents in the isochrone area

In [26]:
# Save the data as pandas data frame
data = pd.DataFrame(apt['iso']["features"])

# Create data frame from column 'properties'
df_pop = pd.DataFrame(data.loc[0, 'properties'])
val = df_pop['total_pop'][0]
print('Number of residents in isochrone area:', f'{val:,.0f}')

{'type': 'FeatureCollection', 'bbox': [8.457399, 47.346561, 8.9652, 47.602965], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 1200.0, 'center': [8.706409857044083, 47.49185026202366], 'total_pop': 468446.0}, 'geometry': {'coordinates': [[[8.457809, 47.419494], [8.460263, 47.419775], [8.462716, 47.420057], [8.464777, 47.420257], [8.466869, 47.420461], [8.469713, 47.420713], [8.472572, 47.420967], [8.475432, 47.421221], [8.478291, 47.421474], [8.47907, 47.421556], [8.479837, 47.421674], [8.480883, 47.421903], [8.483201, 47.42254], [8.48582, 47.423321], [8.48667, 47.423489], [8.495001, 47.423048], [8.495674, 47.421864], [8.49659, 47.421086], [8.497017, 47.420879], [8.497416, 47.420744], [8.500482, 47.419832], [8.502217, 47.419942], [8.502349, 47.420044], [8.507536, 47.426736], [8.510512, 47.427478], [8.510653, 47.427525], [8.511669, 47.427868], [8.513011, 47.428538], [8.515898, 47.429624], [8.519236, 47.430244], [8.526708, 47.429305], [8.528477, 47.429228], [8

## Estimating the purchasing power in the isochrone area

In [11]:
# Simple assumtions:
# 500 CHF per resident and month
# Residents / employee ratio = 2:1
# 20 CHF per employee and working day

# Result
val = ((df_pop['total_pop'][0] * 500) + (df_pop['total_pop'][0] * 0.5 * 20 * 20)) / 10**6
print('Purchasing power per month:',
      f'{val:.1f}',
     'Mio. CHF')

Purchasing power per month: 327.9 Mio. CHF


## Automization of the analysis for multiple of supermarkets

In [20]:
# Client settings
ors = client.Client(key=api_key)

# Parameters for server-request
params_iso = {'profile': 'driving-car',
              'range': [15*60],
              'attributes': ['total_pop']}

# Df to store the data
pop_out = []

try:
    for i in range(0, 10):
        
        print('Preparing isochrone for supermarkt', df[['id', 
                                                        'brand', 
                                                        'bfs_name', 
                                                        'addr:housenumber',
                                                        'addr:postcode']].iloc[i])

        supermarkets = {df['brand'].iloc[i]: {'location': [df['lon'].iloc[i],
                                                        df['lat'].iloc[i]]}}
        # Server request
        for apt in supermarkets.values():
            # Add coords to request parameters
            params_iso['locations'] = [apt['location']]

            # Perform isochrone request
            apt['iso'] = ors.isochrones(**params_iso)
            time.sleep(1)

            # Save the data as pandas data frame
            data = pd.DataFrame(apt['iso']["features"])

            # Create data frame from column 'properties'
            df_pop = pd.DataFrame(data.loc[0, 'properties'])

            # Write values
            pop_out.append([df['id'].iloc[i], 
                            df['brand'].iloc[i],
                            df['bfs_number'].iloc[i], 
                            df['bfs_name'].iloc[i], 
                            df['addr:housenumber'].iloc[i],
                            df['addr:postcode'].iloc[i],
                            df_pop['total_pop'][0],
                            ((df_pop['total_pop'][0] * 500) + (df_pop['total_pop'][0] * 0.5 * 20 * 20)) / 10**6])
            clear_output(wait=True)

except:
    pop_out.append([df['id'].iloc[i], 
                    df['brand'].iloc[i],
                    df['bfs_number'].iloc[i], 
                    df['bfs_name'].iloc[i], 
                    df['addr:housenumber'].iloc[i],
                    df['addr:postcode'].iloc[i],
                    np.NaN,
                    np.NaN])
    clear_output(wait=True)




Preparing isochrone for supermarket Migros:
ID: 119249170
Name: Zürich
Address: 31-35, 8001.0


In [19]:
# Print list
df_pop_out = pd.DataFrame(pop_out, columns=['id', 'brand', 'bfs_number', 'bfs_name', 'addr:housenumber', 'addr:postcode', 'total_pop', 'purchasing_power'])
display(HTML(df_pop_out.to_html(index=False)))

id,brand,bfs_number,bfs_name,addr:housenumber,addr:postcode,total_pop,purchasing_power
280130028,ALDI,3315,Schänis,3,8718.0,76065.0,53.2455
6122906632,Denner,3315,Schänis,14,8718.0,74683.0,52.2781
36726161,Migros,3339,Uznach,25,8730.0,61940.0,43.3580
7389830915,Denner,3339,Uznach,15,8730.0,60275.0,42.1925
39947904,Coop,261,Zürich,1,8001.0,486441.0,340.5087
48932835,Migros,261,Zürich,7,8004.0,482261.0,337.5827
75749133,ALDI,261,Zürich,81,8038.0,320991.0,224.6937
79977755,Coop,261,Zürich,15,8038.0,315037.0,220.5259
83330862,Migros,261,Zürich,3,8038.0,313586.0,219.5102
119249170,Migros,261,Zürich,31-35,8001.0,481913.0,337.3391
